In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'2kwuz9kw'

In [3]:
wandb.init(id='66i4vxlw', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

18297.787

In [5]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(runs, m, wandb=wandb)

CPU times: user 1.18 ms, sys: 381 µs, total: 1.56 ms
Wall time: 8.14 ms


In [6]:
#m = try_load_state_dict(m, torch.load(f"{BESPOKE_ROOT}/models/m.torch"))

In [7]:
#opt = torch.optim.Adam(m.parameters(), lr=3e-4)
opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=.001)

In [8]:
model_stem = "1.29"

In [9]:
%%time
BS = 32
dataloader_trn = TrnLoader(bs=BS)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please 

Got first chunk
CPU times: user 45.4 ms, sys: 249 ms, total: 294 ms
Wall time: 2.14 s
Failed to load run_counter.npy. Retrying...
Failed to load run_counter.npy. Retrying...


Process Process-2:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 128, in make_chunks
    self.chunks_queue.put((img_chunk, aux_chunk, targets_chunk))
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/queues.py", line 83, in put
    if not self._sem.acquire(block, timeout):
KeyboardInterrupt
Process Process-4:
Process Process-3:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py"

In [10]:
"""%%time
m.model_stem = model_stem
rw_evaluator.evaluate()"""

'%%time\nm.model_stem = model_stem\nrw_evaluator.evaluate()'

In [11]:
"""for n, mm in m.backbone.named_modules():
    print(n)
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .99 #.4"""

'for n, mm in m.backbone.named_modules():\n    print(n)\n    if type(mm) == nn.BatchNorm2d: \n        mm.momentum = .99 #.4'

In [12]:
"""freeze_model(m, True)
unfreeze_part_of_model(m, "wps_head")
unfreeze_part_of_model(m, "aux_targets_head")
unfreeze_part_of_model(m, "obsnet")
[p.requires_grad for p in m.parameters()]"""

'freeze_model(m, True)\nunfreeze_part_of_model(m, "wps_head")\nunfreeze_part_of_model(m, "aux_targets_head")\nunfreeze_part_of_model(m, "obsnet")\n[p.requires_grad for p in m.parameters()]'

In [13]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator)

In [14]:
trainer.reload_state()

Loading model from /home/beans/bespoke/models/m.torch. 
Last modified 7 min ago.
Loading opt from /home/beans/bespoke/models/opt.torch. 
Last modified 7 min ago.
Currently at epoch 41.


In [15]:
trainer.train()



 {'avg_unc': -7.17602539, 'wp_angles_i': 0.0055777, 'wp_headings_i': 0.00614603, 'wp_curvatures_i': 0.02837707, 'wp_rolls_i': 4.011e-05, 'wp_zs_i': 0.00092747, 'wp_angles': 0.00171919, 'wp_headings': 0.00216882, 'wp_curvatures': 0.01412296, 'wp_rolls': 0.22336447, 'wp_zs': 0.06637299, 'has_stop': 0.01278865, 'stop_dist': 0.00464189, 'has_lead': 0.00659113, 'lead_dist': 0.0030253, 'lead_speed': 0.36415519, 'dagger_shift': 0.03817441, 'lane_width': 0.07127917, 'rd_is_lined': 0.00205227, 'left_turn': 0.00429839, 'right_turn': 0.00065647, 'unc': 1.05654335, 'grad_norm': 0.03941037, 'logistical/obs_consumed_per_second': 38.90551181, 'logistical/obs_generated_per_second': 91.39370079, 'logistical/slowest_runner_obs_per_sec': 6.0, 'logistical/data_consumption_ratio': 0.4241687, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00240222, 'timing/model forward': 0.20688522, 'timing/calc losses': 0.00670561, 'timing/backwards': 0.61772642, 'timing/get worst': 0.002996

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.





 {'logistical/obs_consumed_per_second': 38.0703125, 'logistical/obs_generated_per_second': 81.0546875, 'logistical/slowest_runner_obs_per_sec': 5.25, 'logistical/data_consumption_ratio': 0.47158455, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00398201, 'timing/model forward': 0.19459784, 'timing/calc losses': 0.00772833, 'timing/backwards': 0.62950427, 'timing/get worst': 0.00463752, 'timing/logging': 0.01447662, 'timing/calc timing': 0.00460644, 'timing/trn update': 0.85953848, 'avg_unc': -7.17938232, 'wp_angles_i': 0.00599876, 'wp_headings_i': 0.0058461, 'wp_curvatures_i': 0.02947396, 'wp_rolls_i': 3.578e-05, 'wp_zs_i': 0.00066841, 'wp_angles': 0.00169736, 'wp_headings': 0.0021484, 'wp_curvatures': 0.01414052, 'wp_rolls': 0.22856963, 'wp_zs': 0.07070541, 'has_stop': 0.01142902, 'stop_dist': 0.00248565, 'has_lead': 0.00811663, 'lead_dist': 0.00153451, 'lead_speed': 0.3175187, 'dagger_shift': 0.06420828, 'lane_width': 0.06736708, 'rd_is_lined': 0.00247



 {'logistical/obs_consumed_per_second': 38.2734375, 'logistical/obs_generated_per_second': 67.7890625, 'logistical/slowest_runner_obs_per_sec': 5.0, 'logistical/data_consumption_ratio': 0.56463793, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00268327, 'timing/model forward': 0.19269691, 'timing/calc losses': 0.00800288, 'timing/backwards': 0.6276778, 'timing/get worst': 0.0036624, 'timing/logging': 0.01566028, 'timing/calc timing': 0.00376729, 'timing/trn update': 0.85415573, 'avg_unc': -7.31268311, 'wp_angles_i': 0.00374586, 'wp_headings_i': 0.00422436, 'wp_curvatures_i': 0.01830135, 'wp_rolls_i': 5.797e-05, 'wp_zs_i': 0.00062806, 'wp_angles': 0.00153221, 'wp_headings': 0.00187111, 'wp_curvatures': 0.01326533, 'wp_rolls': 0.3014378, 'wp_zs': 0.07229304, 'has_stop': 0.01204369, 'stop_dist': 0.00464583, 'has_lead': 0.0075314, 'lead_dist': 0.00195758, 'lead_speed': 0.37538523, 'dagger_shift': 0.04131417, 'lane_width': 0.06455433, 'rd_is_lined': 0.0021501



 {'logistical/obs_consumed_per_second': 37.9765625, 'logistical/obs_generated_per_second': 70.390625, 'logistical/slowest_runner_obs_per_sec': 4.0546875, 'logistical/data_consumption_ratio': 0.53917704, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00419174, 'timing/model forward': 0.19180974, 'timing/calc losses': 0.00841764, 'timing/backwards': 0.63307909, 'timing/get worst': 0.00540197, 'timing/logging': 0.0149362, 'timing/calc timing': 0.00444265, 'timing/trn update': 0.86228372, 'avg_unc': -7.1987915, 'wp_angles_i': 0.00472567, 'wp_headings_i': 0.00530973, 'wp_curvatures_i': 0.03299093, 'wp_rolls_i': 7.315e-05, 'wp_zs_i': 0.00056725, 'wp_angles': 0.00175592, 'wp_headings': 0.0021055, 'wp_curvatures': 0.01439929, 'wp_rolls': 0.22662282, 'wp_zs': 0.06938505, 'has_stop': 0.0148545, 'stop_dist': 0.00334122, 'has_lead': 0.00994316, 'lead_dist': 0.00201358, 'lead_speed': 0.29642724, 'dagger_shift': 0.04113694, 'lane_width': 0.06800437, 'rd_is_lined': 0.00

...



 {'logistical/obs_consumed_per_second': 38.703125, 'logistical/obs_generated_per_second': 62.7890625, 'logistical/slowest_runner_obs_per_sec': 4.71875, 'logistical/data_consumption_ratio': 0.61586645, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.0020248, 'timing/model forward': 0.18976692, 'timing/calc losses': 0.00852104, 'timing/backwards': 0.6203316, 'timing/get worst': 0.00406885, 'timing/logging': 0.01830008, 'timing/calc timing': 0.00392993, 'timing/trn update': 0.84694774, 'avg_unc': -7.43875122, 'wp_angles_i': 0.0022527, 'wp_headings_i': 0.00273098, 'wp_curvatures_i': 0.01959779, 'wp_rolls_i': 6.053e-05, 'wp_zs_i': 0.00057649, 'wp_angles': 0.00125369, 'wp_headings': 0.00155171, 'wp_curvatures': 0.01029131, 'wp_rolls': 0.07048523, 'wp_zs': 0.06110454, 'has_stop': 0.01045573, 'stop_dist': 0.00402003, 'has_lead': 0.00606332, 'lead_dist': 0.00127297, 'lead_speed': 0.26928844, 'dagger_shift': 0.03075053, 'lane_width': 0.05516082, 'rd_is_lined': 0.000



 {'logistical/obs_consumed_per_second': 38.140625, 'logistical/obs_generated_per_second': 71.9375, 'logistical/slowest_runner_obs_per_sec': 5.0, 'logistical/data_consumption_ratio': 0.52823092, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00348264, 'timing/model forward': 0.19299489, 'timing/calc losses': 0.00719456, 'timing/backwards': 0.63229867, 'timing/get worst': 0.00380379, 'timing/logging': 0.02054033, 'timing/calc timing': 0.00418328, 'timing/trn update': 0.86450285, 'avg_unc': -7.44134521, 'wp_angles_i': 0.00392102, 'wp_headings_i': 0.00335475, 'wp_curvatures_i': 0.01769933, 'wp_rolls_i': 1.55e-05, 'wp_zs_i': 0.00059685, 'wp_angles': 0.00112014, 'wp_headings': 0.00159531, 'wp_curvatures': 0.01218084, 'wp_rolls': 0.09515977, 'wp_zs': 0.06508994, 'has_stop': 0.01130114, 'stop_dist': 0.00388009, 'has_lead': 0.00727047, 'lead_dist': 0.00129032, 'lead_speed': 0.2764191, 'dagger_shift': 0.03349439, 'lane_width': 0.05900413, 'rd_is_lined': 0.00219757,



 {'logistical/obs_consumed_per_second': 38.5234375, 'logistical/obs_generated_per_second': 70.0234375, 'logistical/slowest_runner_obs_per_sec': 5.0, 'logistical/data_consumption_ratio': 0.54920908, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00330459, 'timing/model forward': 0.19360186, 'timing/calc losses': 0.00844499, 'timing/backwards': 0.6212291, 'timing/get worst': 0.00422205, 'timing/logging': 0.0234814, 'timing/calc timing': 0.00401516, 'timing/trn update': 0.85830418, 'avg_unc': -7.38818359, 'wp_angles_i': 0.00489362, 'wp_headings_i': 0.00336373, 'wp_curvatures_i': 0.02344601, 'wp_rolls_i': 7.03e-06, 'wp_zs_i': 0.00058434, 'wp_angles': 0.0015194, 'wp_headings': 0.00184749, 'wp_curvatures': 0.01252209, 'wp_rolls': 0.12018263, 'wp_zs': 0.0618397, 'has_stop': 0.01355206, 'stop_dist': 0.00443231, 'has_lead': 0.00773105, 'lead_dist': 0.00162874, 'lead_speed': 0.31266577, 'dagger_shift': 0.0695787, 'lane_width': 0.06770408, 'rd_is_lined': 0.00236253,



 {'logistical/obs_consumed_per_second': 38.15625, 'logistical/obs_generated_per_second': 69.078125, 'logistical/slowest_runner_obs_per_sec': 5.0, 'logistical/data_consumption_ratio': 0.55200192, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00408381, 'timing/model forward': 0.1906789, 'timing/calc losses': 0.00811461, 'timing/backwards': 0.63105731, 'timing/get worst': 0.00399911, 'timing/logging': 0.01635272, 'timing/calc timing': 0.00421985, 'timing/trn update': 0.85851122, 'avg_unc': -7.42974854, 'wp_angles_i': 0.00227799, 'wp_headings_i': 0.00190784, 'wp_curvatures_i': 0.0172016, 'wp_rolls_i': 5.87e-06, 'wp_zs_i': 0.00049921, 'wp_angles': 0.00144364, 'wp_headings': 0.00174959, 'wp_curvatures': 0.01294959, 'wp_rolls': 0.09014028, 'wp_zs': 0.06000209, 'has_stop': 0.01224285, 'stop_dist': 0.0021191, 'has_lead': 0.00837232, 'lead_dist': 0.00160116, 'lead_speed': 0.31099104, 'dagger_shift': 0.04351512, 'lane_width': 0.05675447, 'rd_is_lined': 0.00147651, 



 {'logistical/obs_consumed_per_second': 37.8125, 'logistical/obs_generated_per_second': 73.8203125, 'logistical/slowest_runner_obs_per_sec': 5.09375, 'logistical/data_consumption_ratio': 0.51038214, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00355221, 'timing/model forward': 0.19656646, 'timing/calc losses': 0.00778212, 'timing/backwards': 0.63476898, 'timing/get worst': 0.00351794, 'timing/logging': 0.01667228, 'timing/calc timing': 0.00454062, 'timing/trn update': 0.86740573, 'avg_unc': -7.18789673, 'wp_angles_i': 0.00358754, 'wp_headings_i': 0.00340388, 'wp_curvatures_i': 0.02132801, 'wp_rolls_i': 4.046e-05, 'wp_zs_i': 0.00060987, 'wp_angles': 0.00150195, 'wp_headings': 0.00192245, 'wp_curvatures': 0.01428767, 'wp_rolls': 0.09395391, 'wp_zs': 0.06207442, 'has_stop': 0.01369732, 'stop_dist': 0.00280556, 'has_lead': 0.0073548, 'lead_dist': 0.00295482, 'lead_speed': 0.33425264, 'dagger_shift': 0.04731058, 'lane_width': 0.06554639, 'rd_is_lined': 0.002

In [17]:
trainer.loss_manager.loss_emas

{'has_stop': 0.01079172900048036,
 'stop_dist': 0.0030184585455941175,
 'has_lead': 0.008286640014689494,
 'lead_dist': 0.0018604955010405716,
 'lead_speed': 0.30397185935682847,
 'lane_width': 0.06364512417625082,
 'dagger_shift': 0.05177257378209229,
 'rd_is_lined': 0.0022890943067518216,
 'left_turn': 0.002976004172815129,
 'right_turn': 0.0008691268523601676,
 'td': 1000.0,
 'pitch': 1000.0,
 'yaw': 1000.0,
 'unc': 1.0686791400684255,
 'wp_angles': 0.0016027975107515548,
 'wp_angles_i': 0.004446145144950176,
 'wp_curvatures': 0.013441912221876767,
 'wp_curvatures_i': 0.0279708382396454,
 'wp_headings': 0.0019890282499153544,
 'wp_headings_i': 0.004365436155076691,
 'wp_rolls': 0.11255677601980936,
 'wp_rolls_i': 9.756956819304944e-05,
 'wp_zs': 0.05407107694858163,
 'wp_zs_i': 0.0006262443210981483}

In [23]:
import sys
sys.getsizeof(trainer.loss_manager.loss_emas)

1176

In [16]:
torch.save(trainer.model.state_dict(), f"{BESPOKE_ROOT}/models/m{trainer.model_stem}_e{trainer.current_epoch}.torch")

In [16]:
m.training

Error in callback <function _WandbInit._resume_backend at 0x7f62ef023160> (for pre_run_cell):


Exception: The wandb backend process has shutdown

True

Error in callback <function _WandbInit._pause_backend at 0x7f62e3b863a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
